## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["daily_address_summary"],
    range_spec="@20241030:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-05 15:37:41 [info     ] Rendering query                filename=querybuilder.py lineno=40 model=daily_address_summary template=transaction_fees
2024-12-05 15:37:41 [info     ] Rendering query                filename=querybuilder.py lineno=40 model=daily_address_summary template=daily_address_summary
2024-12-05 15:37:41 [info     ] Rendering query                filename=querybuilder.py lineno=40 model=contract_creation template=contract_creation_traces
2024-12-05 15:37:41 [info     ] Loaded 2 python modules with intermediate model definitions. filename=registry.py lineno=62
2024-12-05 15:37:41 [info     ] loaded vault from .env file    filename=vault.py lineno=32
2024-12-05 15:37:41 [debug    ] loaded vault: 18 items         filename=vault.py lineno=76
2024-12-05 15:37:41 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18
2024-12-05 15:37:42 [info     ] Loaded OP chains metadata from /Users/pedro/github/ethereum-optimism/op-anal

## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory


execute_model_in_memory(
    duckdb_client=duckdb_client,
    model="daily_address_summary",
    data_reader=batch,
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
#
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-05 15:38:04 [info     ] Executing model...             filename=testutils.py lineno=226
2024-12-05 15:38:04 [info     ] duckdb dataset='ingestion/blocks_v1' using 23/23 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/blocks_v1/chain=op/dt=2024-10-30/000127322000.parquet filename=reader.py lineno=63
2024-12-05 15:38:05 [info     ] registered view: 'ingestion_blocks_v1' using 23 parquet paths filename=client.py lineno=53
2024-12-05 15:38:05 [info     ] duckdb dataset='ingestion/transactions_v1' using 23/23 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-10-30/000127322000.parquet filename=reader.py lineno=63
2024-12-05 15:38:06 [info     ] registered view: 'ingestion_transactions_v1' using 23 parquet paths filename=client.py lineno=53
dict_keys(['summary_v1'])


┌───────────────────────────┐
│           name            │
│          varchar          │
├───────────────────────────┤
│ daily_address_summary     │
│ ingestion_blocks_v1       │
│ ingestion_transactions_v1 │
│ summary_v1                │
│ transaction_fees          │
└───────────────────────────┘

## Verify model results

In [3]:
duckdb_client.sql("SELECT * FROM summary_v1 LIMIT 10")

┌────────────┬─────────┬──────────┬────────────────────────────────────────────┬────────┬────────────────┬───────────┬───────────────────┬──────────────────┬──────────────────┬────────────────────┬───────────┬───────────┬────────────────────┬─────────────────────┬─────────────────────┬───────────────────┬───────────────────┬─────────────────┬─────────────────────────┬────────────────┬─────────────────┬─────────────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬────────────────────────┬───────────────────────┬───────────────────────┬─────────────────────────┬────────────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬────────────────────────┬────────────────────────────┬────────────────────────┬──────────────────────┐
│     dt     │  chain  │ chain_id │                  address                   │ tx_cnt │ success_tx_cnt │ block_cnt │ success_block_cnt │ block_number_min │ block_number_max

### You can also convert the results to dataframes to inspect them in more familiar ways

In [4]:
duckdb_client.sql("SELECT * FROM summary_v1 LIMIT 10").pl().head()

dt,chain,chain_id,address,tx_cnt,success_tx_cnt,block_cnt,success_block_cnt,block_number_min,block_number_max,active_block_range,nonce_min,nonce_max,active_nonce_range,block_timestamp_min,block_timestamp_max,active_time_range,active_hours_ucnt,to_address_ucnt,success_to_address_ucnt,method_id_ucnt,l2_gas_used_sum,success_l2_gas_used_sum,l1_gas_used_sum,success_l1_gas_used_sum,tx_fee_sum_eth,success_tx_fee_sum_eth,l2_fee_sum_eth,l2_base_fee_sum_eth,l2_priority_fee_sum_eth,l2_base_legacy_fee_sum_eth,l1_fee_sum_eth,l1_base_fee_sum_eth,l1_blob_fee_sum_eth,l2_gas_price_avg_gwei,l2_base_price_avg_gwei,l2_priority_price_avg_gwei,l1_base_price_avg_gwei,l1_blob_fee_avg_gwei
date,str,i32,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,u32,u32,u32,i64,i64,i64,i64,"decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]"
2024-10-30,"""op""",10,"""0xaa971288ea224d94014d7cafd31f…",6099,5980,4637,4584,127323823,127367011,43189,263880,269978,6099,1730246423,1730332799,86376,24,3,3,5,2011866171,1991937468,15349044,15026564,0.0295394861834670890,0.0291519833502175670,0.0271367522283079090,0.0000009284876573830,0.0271358237406505260,0.0000000000000000000,0.0024027339551591800,0.0010233699604265900,0.0013790109078847010,0.0134883490,0.0000004620,0.0134878870,12.7555395470,1.4173502350
2024-10-30,"""op""",10,"""0x80c67432656d59144ceff962e8fa…",17702,17702,17633,17633,127323813,127367008,43196,1057544,1075245,17702,1730246403,1730332793,86390,24,8349,8349,2,372007232,372007232,28334322,28334322,0.0047049174966654290,0.0047049174966654290,0.0004465811645339340,0.0000001724861339340,0.0004464086784000000,0.0000000000000000000,0.0042583363321314950,0.0018015727516364060,0.0024567633068750720,0.0012004640,0.0000004640,0.0012000000,12.1642819940,1.3678589040
2024-10-30,"""op""",10,"""0x7cda773fc37b31d0f5fb309b5e07…",1,1,1,1,127344003,127344003,1,211,211,1,1730286783,1730286783,0,1,1,1,1,211807,211807,1600,1600,0.0000003574901318760,0.0000003574901318760,0.0000000212499608890,0.0000000000692608890,0.0000000211807000000,0.0000000000000000000,0.0000003362401709870,0.0000000940855941590,0.0000002421545768280,0.0001003270,0.0000003270,0.0001000000,11.2499514730,2.3876106580
2024-10-30,"""op""",10,"""0x9a9c86edafb44f065c1aa76f03e9…",158,158,158,158,127327802,127365302,37501,48638,48795,158,1730254381,1730329381,75000,22,1,1,3,4588854,4588854,252800,252800,0.0000519553863943680,0.0000519553863943680,0.0000045909825292270,0.0000000021285292270,0.0000045888540000000,0.0000000000000000000,0.0000473644038651410,0.0000190663862292320,0.0000282980176359900,0.0010004640,0.0000004640,0.0010000000,14.4290858240,1.7659148270
2024-10-30,"""op""",10,"""0x28a3b352f53804c6afb1c08093ea…",10,10,10,10,127343963,127344134,172,0,9,10,1730286703,1730287045,342,1,3,3,3,2127359,2127359,53260,53260,0.0000137701522657060,0.0000137701522657060,0.0000003201331506270,0.0000000006925036370,0.0000003194406469900,0.0000000000000000000,0.0000134500191150790,0.0000029457010747910,0.0000105028255853550,0.0001504840,0.0000003260,0.0001501580,10.5812021380,3.1109698420
